In [1]:
import requests
import pandas as pd
from pprint import pprint
import time
import config

# API request setup
url = "https://realtor-data3.p.rapidapi.com/realtor/SearchSold"
headers = {
    'x-rapidapi-key': config.RAPIDAPI_KEY,
    'x-rapidapi-host': config.RAPIDAPI_HOST
}


In [2]:
# List to hold Property Values
property_list = []

# Loop through multiple pages
for page in range(1, 15):
    querystring = {"location": "St. Paul, MN", "sort": "best_match", "property_type": "single_family_home", "page": page,  "page_size": 120 }

    response = requests.get(url, headers=headers, params=querystring)
        # Error handling for failed requests
    if response.status_code != 200:
        print(f"⚠️ Error fetching page {page}: {response.status_code}")
        continue
    data = response.json()
#Extract Property Values
    for item in data.get("data", []):
      # Safely access nested dictionary keys, providing default None values if a key is missing
        location_data = item.get("location", {})
        address_data = location_data.get("address", {})
        coordinate_data = address_data.get("coordinate", {}) # Ensure coordinate_data is a dict or None

        # Use .get() with a default value for 'lat' and 'lon' in case coordinate_data is None
        latitude = coordinate_data.get("lat") if coordinate_data is not None else None
        longitude = coordinate_data.get("lon") if coordinate_data is not None else None

        property_list.append({
        "list_date": item.get("list_date"),
        "list_price": item.get("list_price"),
        "sold_date": item.get("description", {}).get("sold_date"),
        "sold_price": item.get("description", {}).get("sold_price"),
        "beds": item.get("description", {}).get("beds"),
        "baths": item.get("description", {}).get("baths_consolidated"),
        "sqft": item.get("description", {}).get("sqft"),
        "lot_sqft": item.get("description", {}).get("lot_sqft"),
        "city": item.get("location", {}).get("address", {}).get("city"),
        "street": item.get("location", {}).get("address", {}).get("line"),
        "zip": item.get("location", {}).get("address", {}).get("postal_code"),
        "latitude": latitude,
        "longitude": longitude,
    })
    # Avoid hitting rate limits
    time.sleep(1)

# Create DataFrame
df = pd.DataFrame(property_list)

# Display first few rows
df

,list_date,list_price,sold_date,sold_price,beds,baths,sqft,lot_sqft,city,street,zip,latitude,longitude
0,2025-01-23T14:51:53.000000Z,500000.0,2025-02-28,505003,3.0,2,1636.0,5009.0,Saint Paul,2098 Pinehurst Ave,55116,44.918628,-93.189055
1,2024-09-03T13:57:07.000000Z,189900.0,2024-12-11,189900,2.0,1,832.0,4792.0,Saint Paul,536 Edmund Ave,55103,44.958192,-93.122838
2,2024-12-20T04:16:16.000000Z,275000.0,2025-01-10,266800,3.0,1,1297.0,6534.0,Saint Paul,967 California Ave W,55117,44.991249,-93.140914
3,2024-12-01T23:45:19.000000Z,324900.0,2025-01-31,325000,3.0,1.5,2185.0,6055.0,Saint Paul,627-629 Minnehaha Ave E,55130,44.963264,-93.074138
4,2024-11-19T22:04:37.000000Z,214900.0,2025-01-09,210000,2.0,1,926.0,5001.0,Saint Paul,1631 Woodbridge St,55117,44.990397,-93.108765
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,2024-12-30T20:19:30.000000Z,324500.0,2025-01-31,337000,4.0,2,1697.0,7536.0,North Saint Paul,2215 Shawnee Dr,55109,45.024647,-93.006823
1469,2025-03-07T02:29:44.000000Z,215000.0,2025-04-29,180000,4.0,1.5,1682.0,4966.0,North Saint Paul,2663 6th Ave E,55109,45.012383,-92.987993
1470,None,NaN,2025-04-15,225000,5.0,2.5,1854.0,11308.0,Saint Paul,2454 Ripley Ave,55109,44.995915,-92.996645
1471,None,NaN,2025-03-11,90000,2.0,1,801.0,5249.0,Saint Paul,2567 3rd Ave E,55109,45.009610,-92.992046


In [3]:
#Convert List Date to Datetime
df["list_date"] = pd.to_datetime(df["list_date"])
#Remove Timestamp
df["list_date"] = df["list_date"].dt.date

In [4]:
#Format list_price, beds, baths, sq_ft, lot_sqft
df["list_price"] = df["list_price"].astype('Int64')
df["beds"] = df["beds"].astype('Int64')
df["sqft"] = df["sqft"].astype('Int64')
df["lot_sqft"] = df["lot_sqft"].astype('Int64')
df["baths"] = df["baths"].astype(str).str.replace("+", "").replace("None", None).astype(float)
df


,list_date,list_price,sold_date,sold_price,beds,baths,sqft,lot_sqft,city,street,zip,latitude,longitude
0,2025-01-23,500000,2025-02-28,505003,3,2.0,1636,5009,Saint Paul,2098 Pinehurst Ave,55116,44.918628,-93.189055
1,2024-09-03,189900,2024-12-11,189900,2,1.0,832,4792,Saint Paul,536 Edmund Ave,55103,44.958192,-93.122838
2,2024-12-20,275000,2025-01-10,266800,3,1.0,1297,6534,Saint Paul,967 California Ave W,55117,44.991249,-93.140914
3,2024-12-01,324900,2025-01-31,325000,3,1.5,2185,6055,Saint Paul,627-629 Minnehaha Ave E,55130,44.963264,-93.074138
4,2024-11-19,214900,2025-01-09,210000,2,1.0,926,5001,Saint Paul,1631 Woodbridge St,55117,44.990397,-93.108765
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,2024-12-30,324500,2025-01-31,337000,4,2.0,1697,7536,North Saint Paul,2215 Shawnee Dr,55109,45.024647,-93.006823
1469,2025-03-07,215000,2025-04-29,180000,4,1.5,1682,4966,North Saint Paul,2663 6th Ave E,55109,45.012383,-92.987993
1470,NaT,<NA>,2025-04-15,225000,5,2.5,1854,11308,Saint Paul,2454 Ripley Ave,55109,44.995915,-92.996645
1471,NaT,<NA>,2025-03-11,90000,2,1.0,801,5249,Saint Paul,2567 3rd Ave E,55109,45.009610,-92.992046


In [5]:
duplicates = df[df.duplicated()]
print(duplicates)

       list_date  list_price   sold_date  sold_price  beds  baths  sqft  \
122   2025-02-03      349900  2025-03-11      356500     3    2.5  1985   
123          NaT        <NA>  2024-12-10      269000     3    1.0  1048   
124          NaT        <NA>  2024-12-11      220000     2    1.0  1368   
125   2024-12-03      219900  2025-01-28      219900     3    1.0  1705   
128   2025-02-13      200000  2025-03-10      225000     3    1.5  1135   
...          ...         ...         ...         ...   ...    ...   ...   
1448         NaT        <NA>  2025-03-13      199800     4    3.0  2384   
1449         NaT        <NA>  2025-03-28      665000     5    2.5  4192   
1450         NaT        <NA>  2025-05-05      380000     4    5.5  2734   
1451         NaT        <NA>  2025-05-02      310000     4    3.5  2440   
1452         NaT        <NA>  2025-04-07      470000     3    2.5  5233   

      lot_sqft        city              street    zip   latitude  longitude  
122       9060  Saint

In [ ]:
# Save as CSV
df.to_csv("st_paul_sold_properties.csv", index=False)

In [6]:
#Remove duplicates
df_cleaned = df.drop_duplicates()
df_cleaned.to_csv("cleaned_st_paul_properties.csv", index=False)